In [48]:
# imports
import os
import requests
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [49]:
fs_api_key = os.environ["foursq_api"]

In [50]:
enlabici_df = pd.read_csv('../data/enlabici-mend.csv')

foursq_results = []

# Loop through each row in the enlabici_df
for index, row in enlabici_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Define the base URL of the Foursquare API
    url = 'https://api.foursquare.com/v3/places/search'

    # Define the parameters (13000 for Dining & Drinking; 10027 for Art and Entertainment > Museum)
    params = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'categories': '13000,10027',
        'fields': 'name,categories,distance,rating,popularity,price',
        'open_at': '1T1931',
        'sort': 'RATING',
        'limit': 20
    }
  
    # Define the headers
    headers = {'Accept': 'application/json'}
    headers['Authorization'] = fs_api_key  

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
    except HTTPError as errh:
        print ("Http Error:",errh)
    except ConnectionError as errc:
        print ("Error Connecting:",errc)
    except Timeout as errt:
        print ("Timeout Error:",errt)
    except TooManyRedirects as errr:
        print ("Too Many Redirects:",errr)
    except RequestException as err:
        print ("Something Else:",err)
    else:
        data = response.json()
        
    places = data.get('results')
    for place in places:
        place['latitude'] = latitude
        place['longitude'] = longitude
        foursq_results.append(place)


In [51]:
fs_df = pd.DataFrame(foursq_results)
fs_df

,categories,distance,name,popularity,price,rating,latitude,longitude
0,"[{'id': 13070, 'name': 'Argentinian Restaurant...",523,Azafran,0.998870,3.0,8.9,-32.884090,-68.848970
1,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",895,BRÖD Panadería,0.991713,1.0,8.9,-32.884090,-68.848970
2,"[{'id': 13035, 'name': 'Coffee Shop', 'short_n...",514,Famiglia Perin,0.987067,1.0,8.8,-32.884090,-68.848970
3,"[{'id': 13070, 'name': 'Argentinian Restaurant...",865,Ceibo Restaurante,0.944375,2.0,8.6,-32.884090,-68.848970
4,"[{'id': 13070, 'name': 'Argentinian Restaurant...",194,Anna Bistro,0.992843,3.0,8.5,-32.884090,-68.848970
...,...,...,...,...,...,...,...,...
315,"[{'id': 13065, 'name': 'Restaurant', 'short_na...",797,Dorrego Sur Sandwich,0.501128,NaN,NaN,-32.924682,-68.842504
316,"[{'id': 13064, 'name': 'Pizzeria', 'short_name...",822,Gallego Pizza,0.956247,1.0,NaN,-32.924682,-68.842504
317,"[{'id': 13065, 'name': 'Restaurant', 'short_na...",850,Cardamomo Resto,0.248083,NaN,NaN,-32.924682,-68.842504
318,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",900,San Francisco,0.775113,1.0,NaN,-32.870709,-68.858351


In [52]:
fs_df.to_csv('../data/foursquare_df_orig.csv', index=False)

In [53]:
# Check missing coordinate -32.9434,-68.818481
url = 'https://api.foursquare.com/v3/places/search'

params = {
    'll': '-32.9434,-68.818481',
    'radius': 1000,
    'categories': '13000,10027',
    'fields': 'name,categories,distance,rating,popularity,price',
    'open_at': '1T1931',
    'sort': 'RATING',
    'limit': 20
}
headers = {'Accept': 'application/json'}
headers['Authorization'] = fs_api_key  

response = requests.get(url, params=params, headers=headers)

data = response.json()
add_place = data.get('results')

In [54]:
print(response)
print(data)
print(add_place)

<Response [200]>
{'results': [], 'context': {'geo_bounds': {'circle': {'center': {'latitude': -32.9434, 'longitude': -68.818481}, 'radius': 1000}}}}
[]


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice.

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [55]:
yelp_api_key = os.environ["yelp_api"]

In [56]:
enlabici_df = pd.read_csv('../data/enlabici-mend.csv')

yelp_results = []

# Loop through each row in the enlabici_df
for index, row in enlabici_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Define the base URL of the Foursquare API
    url = 'https://api.yelp.com/v3/businesses/search'
    
    # Define the parameters 
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'term': 'restaurants',
        'radius': 1000,
        'categories': 'restaurants,All',
        'sort_by': 'rating',
        'limit': 20
    }
    
    # Define the headers
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {yelp_api_key}'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
    except HTTPError as errh:
        print ("Http Error:",errh)
    except ConnectionError as errc:
        print ("Error Connecting:",errc)
    except Timeout as errt:
        print ("Timeout Error:",errt)
    except TooManyRedirects as errr:
        print ("Too Many Redirects:",errr)
    except RequestException as err:
        print ("Something Else:",err)
    else:
        data = response.json()
        
    places = data.get('businesses')
    for place in places:
        place['latitude'] = latitude
        place['longitude'] = longitude
        yelp_results.append(place)

In [57]:
yp_df = pd.DataFrame(yelp_results)
yp_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,HtAhkhT4ITrDFSQxqbbucQ,cachitas-mendoza,Cachita's,https://s3-media1.fl.yelpcdn.com/bphoto/cli340...,False,https://www.yelp.com/biz/cachitas-mendoza?adju...,7,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,"{'latitude': -32.8888128760799, 'longitude': -...",[],$$,"{'address1': 'Av. Sarmiento 784', 'address2': ...",+542614258958,+54 261 425-8958,526.499634,-32.884090,-68.848970
1,I3yljXkYDFRAZ3TuObhC5w,la-barra-mendoza,La Barra,https://s3-media3.fl.yelpcdn.com/bphoto/gVwVZO...,False,https://www.yelp.com/biz/la-barra-mendoza?adju...,5,"[{'alias': 'argentine', 'title': 'Argentine'}]",5.0,"{'latitude': -32.8898057430266, 'longitude': -...",[],$$$,"{'address1': 'Av. Belgrano 1086', 'address2':...",+541156541950,+54 11 5654-1950,645.835522,-32.884090,-68.848970
2,V2sDzIU3rKFkA7r_8-POuQ,unión-mendoza,Unión,https://s3-media2.fl.yelpcdn.com/bphoto/qkh9dz...,False,https://www.yelp.com/biz/uni%C3%B3n-mendoza?ad...,5,"[{'alias': 'cafeteria', 'title': 'Cafeteria'}]",5.0,"{'latitude': -32.8886898, 'longitude': -68.849...",[],NaN,"{'address1': 'Av. Sarmiento 777', 'address2': ...",+542644299009,+54 264 429-9009,509.795344,-32.884090,-68.848970
3,p_nchEuqalCPSzYjR2QCJg,cordillera-vinos-y-fuegos-mendoza-2,Cordillera Vinos y Fuegos,https://s3-media1.fl.yelpcdn.com/bphoto/5zuotZ...,False,https://www.yelp.com/biz/cordillera-vinos-y-fu...,13,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': -32.8895176468452, 'longitude': -...",[],$$$,"{'address1': 'Belgrano 1028', 'address2': '', ...",+542614239704,+54 261 423-9704,612.792296,-32.884090,-68.848970
4,w4Mt3rz9-pcSKMYcnAeqrg,silla-14-mendoza,Silla 14,https://s3-media2.fl.yelpcdn.com/bphoto/hkIT9V...,False,https://www.yelp.com/biz/silla-14-mendoza?adju...,8,"[{'alias': 'cafeteria', 'title': 'Cafeteria'}]",4.5,"{'latitude': -32.8923837459877, 'longitude': -...",[],$$,"{'address1': 'San Lorenzo 656', 'address2': ''...",+542614238837,+54 261 423-8837,922.494041,-32.884090,-68.848970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,EuVG8YRvW9zDGb5hx7SOpg,gula-gula-la-pizza-delivery-godoy-cruz,Gula Gula - la Pizza - Delivery,,False,https://www.yelp.com/biz/gula-gula-la-pizza-de...,0,"[{'alias': 'pizza', 'title': 'Pizza'}]",0.0,"{'latitude': -32.917782, 'longitude': -68.832092}",[],NaN,"{'address1': 'Panamericana 2265', 'address2': ...",+542614399990,+54 261 439-9990,1238.203183,-32.924682,-68.842504
274,1cZzCYzqkGGmmRwDmLcyXQ,aldo-pizzeria-sandwicheria-restaurante-deliver...,Aldo Pizzeria Sandwicheria Restaurante - Delivery,,False,https://www.yelp.com/biz/aldo-pizzeria-sandwic...,1,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': -32.877064, 'longitude': -68.856554}",[],NaN,"{'address1': 'Suipacha 698', 'address2': None,...",+542614295263,+54 261 429-5263,726.297313,-32.870709,-68.858351
275,lrun9tbFJxnr3YdqDe4k4A,la-paulina-pizzeria-delivery-mendoza,la Paulina Pizzeria - Delivery,,False,https://www.yelp.com/biz/la-paulina-pizzeria-d...,0,"[{'alias': 'pizza', 'title': 'Pizza'}]",0.0,"{'latitude': -32.873981, 'longitude': -68.849019}",[],NaN,"{'address1': 'Cnel Rodríguez 2406', 'address2'...",+542614299889,+54 261 429-9889,944.414799,-32.870709,-68.858351
276,jhjx3L8t-YwP23cUjvZ5vA,pizzas-artesanales-amarello-mendoza,Pizzas Artesanales Amarello,,False,https://www.yelp.com/biz/pizzas-artesanales-am...,0,"[{'alias': 'pizza', 'title': 'Pizza'}]",0.0,"{'latitude': -32.871051, 'longitude': -68.855062}",[],NaN,"{'address1': 'Paso D L Andes 2566', 'address2'...",+542614298358,+54 261 429-8358,309.511561,-32.870709,-68.858351


In [58]:
yp_df.to_csv('../data/yelp_df_orig.csv', index=False)

# Comparing Results

**Which API provided you with more complete data? Provide an explanation.**

Overall, the Foursquare API seems better in terms of completeness of data, ease of use, and ability to select data
* With a limit of 20 for each latitude/longitude row, Foursquare pulled 320 total results while Yelp pulled 278. 
* Looking at the column with the most nulls, Foursquare has a non-null value for 73% of the entries (ratings 233 out of 320). Yelp has a value for 50% of the entries (price 140 out of 278)
* Foursquare rating column ranges from 0-10, while Yelp ranges from 0-5. When sorting by rating, Foursquare results show more distinction between entries.
* Foursquare lets you finetune the fields returned a bit more easily than Yelp

Yelp provides more data columns but completeness and the ability to finetune is more useful rather than getting lots of data for the sake of it. However, a point in favour of Yelp is that it covered all 22 bike stations. Foursquare returned results for 21 latitude/longitude coordinates. Submitting a request for the missing coordinate returned no response. 

In [59]:
fs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categories  320 non-null    object 
 1   distance    320 non-null    int64  
 2   name        320 non-null    object 
 3   popularity  320 non-null    float64
 4   price       264 non-null    float64
 5   rating      233 non-null    float64
 6   latitude    320 non-null    float64
 7   longitude   320 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 20.1+ KB


In [60]:
yp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             278 non-null    object 
 1   alias          278 non-null    object 
 2   name           278 non-null    object 
 3   image_url      278 non-null    object 
 4   is_closed      278 non-null    bool   
 5   url            278 non-null    object 
 6   review_count   278 non-null    int64  
 7   categories     278 non-null    object 
 8   rating         278 non-null    float64
 9   coordinates    278 non-null    object 
 10  transactions   278 non-null    object 
 11  price          140 non-null    object 
 12  location       278 non-null    object 
 13  phone          278 non-null    object 
 14  display_phone  278 non-null    object 
 15  distance       278 non-null    float64
 16  latitude       278 non-null    float64
 17  longitude      278 non-null    float64
dtypes: bool(1)

**Get the top 10 restaurants according to their rating**

In [61]:
# FOURSQUARE TOP 10 

# create a copy of fs_df
fs_df1 = fs_df.copy()

# drop duplicates between the columns: name, rating, popularity, price; keep first instance
fs_df1_unique = fs_df1.drop_duplicates(subset=['name','rating','popularity','price'], keep='first')

# show top 10 rows ordered by 'rating' column
fs_top_10 = fs_df1_unique.nlargest(10, 'rating')

print(fs_top_10[['name','rating','popularity']])

                             name  rating  popularity
0                         Azafran     8.9    0.998870
1                  BRÖD Panadería     8.9    0.991713
2                  Famiglia Perin     8.8    0.987067
3               Ceibo Restaurante     8.6    0.944375
4                     Anna Bistro     8.5    0.992843
94                   Bodegas CARO     8.5    0.956429
121                     Calendula     8.4    0.956806
25                      Starbucks     8.3    0.989076
44              Cuarto Intermedio     8.2    0.804746
5    Brillat Savarin - Pastelería     8.1    0.945756


In [62]:
# YELP TOP 10

# create a copy of yp_df
yp_df1 = yp_df.copy()

# drop duplicates between the columns: name, rating, review_count; keep first instance
yp_df1_unique = yp_df1.drop_duplicates(subset=['name', 'rating','review_count'], keep='first')

# show top 10 rows ordered by 'rating' column
yp_top_10 = yp_df1_unique.nlargest(10, 'rating')

print(yp_top_10[['name', 'rating', 'review_count']])

                  name  rating  review_count
0            Cachita's     5.0             7
1             La Barra     5.0             5
2                Unión     5.0             5
6    Tasting Patagonia     5.0             4
11        Caché Bistro     5.0             2
12     Santa y Pecador     5.0             2
51  Burger Social Club     5.0             1
52      Café Plazoleta     5.0             1
53   Tasca de La Plaza     5.0             1
61   Cantina la Rambla     5.0             1


The above results also highlight where Yelp falls short. The top 10 Yelp places based on ratings don't actually have many reviews. They could just be highly rated (for now) because they're fairly new.

For example, "Anna Bistro" falls under the top 10 in the Foursquare results, however, for Yelp it doesn't make the cut because it has a 4.5 rating BUT 45 reviews. All top 10 results for Yelp have under 10 reviews.